In [1]:
pip install mediapipe opencv-python numpy


  Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached jax-0.4.30-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.30-cp311-cp311-win_amd64.whl.metadata (1.1 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.4.7-py3-none-win_amd64.whl.metadata (1.4 kB)
Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl (50.8 MB)
Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl (413 kB)
Using cached sounddevice-0.4.7-py3-none-win_amd64.whl (200 kB)
Using cached jax-0.4.30-py3-none-any.whl (2.0 MB)
   ---------------------------------------- 0.0/51.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/51.9 MB 960.0 kB/s eta 0:00:55
   ---------------------------------------- 0.1/51.9 MB 1.3 MB/s eta 0:00:40
   ---------------------------------------- 0.1/51.9 MB 1.3 MB/s eta 0:00:40
   ---------------------------------------- 0.2/51.9 MB 1.4 MB/s eta 0:00:38
   ------------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   -- ------------------------------------- 2.9/51.9 MB 880.9 kB/s eta 0:00:56
   -- ------------------------------------- 2.9/51.9 MB 863.6 kB/s eta 0:00:57
   -- ------------------------------------- 3.0/51.9 MB 850.1 kB/s eta 0:00:58
   -- ------------------------------------- 3.0/51.9 MB 850.1 kB/s eta 0:00:58
   -- ------------------------------------- 3.0/51.9 MB 837.9 kB/s eta 0:00:59
   -- ------------------------------------- 3.0/51.9 MB 829.1 kB/s eta 0:01:00
   -- ------------------------------------- 3.0/51.9 MB 817.1 kB/s eta 0:01:00
   -- ------------------------------------- 3.0/51.9 MB 817.1 kB/s eta 0:01:00
   -- ------------------------------------- 3.0/51.9 MB 809.5 kB/s eta 0:01:01
   -- ------------------------------------- 3.0/51.9 MB 798.3 kB/s eta 0:01:02
   -- ------------------------------------- 3.1/51.9 MB 791.2 kB/s eta 0:01:02
   -- ------------------------------------- 3.1/51.9 MB 786.9 kB/s eta 0:01:03
   -- ------------------------------------- 3.1/51.9

In [2]:
import cv2
import mediapipe as mp
import numpy as np

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Video capture (0 for webcam or use a video file path)
cap = cv2.VideoCapture(0)

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# Define counters and flags
counter = 0
stage = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Perform pose detection
    results = pose.process(image)
    
    # Revert the image to BGR for rendering
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark
        
        # Get coordinates for knee, hip, and ankle (for squats)
        hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
               landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        
        # Calculate angle
        angle = calculate_angle(hip, knee, ankle)
        
        # Squat detection logic
        if angle > 160:
            stage = "up"
        if angle < 100 and stage == "up":
            stage = "down"
            counter += 1
            print(f"Squat count: {counter}")
        
        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Display angle and count on the image
        cv2.putText(image, f'Angle: {int(angle)}', (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'Count: {counter}', (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    except:
        pass
    
    # Display the image
    cv2.imshow('Pose Detection', image)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\pooja\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Squat count: 1
Squat count: 2


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Video capture (0 for webcam or use a video file path)
cap = cv2.VideoCapture(0)

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# Define counters, flags, and exercise labels
counter = { 'Squat': 0, 'Push-up': 0, 'Stand': 0, 'Sit': 0, 'Jump': 0 }  # Counter for each exercise
stage = None
exercise_label = "Not Detected"

# Define thresholds for recognizing activities
activity_thresholds = {
    'squat_knee_angle': (60, 160),      # Squat detected if knee angle is between 60 and 160 degrees
    'pushup_elbow_angle': (60, 160),    # Push-up detected if elbow angle is between 60 and 160 degrees
    'standing_knee_angle': (160, 180),  # Standing detected if knee angle is between 160 and 180 degrees
    'sitting_hip_angle': (60, 120),     # Sitting detected if hip angle is between 60 and 120 degrees
    'jumping_hip_angle': (150, 180)     # Jumping detected if hip angle is between 150 and 180 degrees
}

# Function to draw exercise label on the image
def draw_exercise_label(image, label):
    cv2.putText(image, f'Exercise: {label}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

# Flag to control video capture loop
is_running = True

while is_running:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Perform pose detection
    results = pose.process(image)
    
    # Revert the image to BGR for rendering
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Extract landmarks and check if they exist
    try:
        landmarks = results.pose_landmarks.landmark
        
        # Get coordinates for relevant body parts
        hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
               landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        
        # Calculate angles for exercise recognition
        knee_angle = calculate_angle(hip, knee, ankle)
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        hip_angle = calculate_angle(shoulder, hip, knee)
        
        # Reset exercise label each frame
        exercise_label = "Not Detected"
        
        # Detect Squats based on knee angle
        if activity_thresholds['squat_knee_angle'][0] < knee_angle < activity_thresholds['squat_knee_angle'][1]:
            if stage == "up":
                stage = "down"
                counter['Squat'] += 1
                exercise_label = "Squat"
                print(f"Squat count: {counter['Squat']}")
            elif knee_angle > 160:
                stage = "up"
        
        # Detect Push-ups based on elbow angle
        elif activity_thresholds['pushup_elbow_angle'][0] < elbow_angle < activity_thresholds['pushup_elbow_angle'][1]:
            if stage == "up":
                stage = "down"
                counter['Push-up'] += 1
                exercise_label = "Push-up"
                print(f"Push-up count: {counter['Push-up']}")
            elif elbow_angle > 160:
                stage = "up"
        
        # Detect Standing based on knee angle
        elif activity_thresholds['standing_knee_angle'][0] < knee_angle < activity_thresholds['standing_knee_angle'][1]:
            counter['Stand'] += 1
            exercise_label = "Standing"
        
        # Detect Sitting based on hip angle
        elif activity_thresholds['sitting_hip_angle'][0] < hip_angle < activity_thresholds['sitting_hip_angle'][1]:
            counter['Sit'] += 1
            exercise_label = "Sitting"
        
        # Detect Jumping based on hip angle and vertical movement
        elif activity_thresholds['jumping_hip_angle'][0] < hip_angle < activity_thresholds['jumping_hip_angle'][1]:
            # Add additional condition to check for upward movement indicating a jump
            if results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].visibility > 0.8:
                counter['Jump'] += 1
                exercise_label = "Jumping"
        
        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Draw body part labels
        body_parts = {
            'Hip': hip,
            'Knee': knee,
            'Ankle': ankle,
            'Shoulder': shoulder,
            'Elbow': elbow,
            'Wrist': wrist
        }
        
        for part, coord in body_parts.items():
            coord = tuple(np.multiply(coord, [image.shape[1], image.shape[0]]).astype(int))
            cv2.putText(image, part, coord, cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Display angles and exercise label on the image
        cv2.putText(image, f'Knee Angle: {int(knee_angle)}', (10, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f'Elbow Angle: {int(elbow_angle)}', (10, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f'Hip Angle: {int(hip_angle)}', (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f'Squat Count: {counter["Squat"]}', (10, 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f'Push-up Count: {counter["Push-up"]}', (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f'Standing Count: {counter["Stand"]}', (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f'Sitting Count: {counter["Sit"]}', (10, 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f'Jumping Count: {counter["Jump"]}', (10, 160), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
        draw_exercise_label(image, exercise_label)
    
    except Exception as e:
        print(f"Error: {e}")
        # Draw landmarks even if exercise detection fails
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    # Display the image
    cv2.imshow('Pose Detection', image)
    
    # Check if 'q' key was pressed or window was closed
    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        is_running = False

# Release resources
cap.release()
cv2.destroyAllWindows()

C:\Users\pooja\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Video capture (0 for webcam or use a video file path)
cap = cv2.VideoCapture(0)

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

# Function to determine if the body is stable or moving
def is_body_stable(landmarks, prev_landmarks, threshold=0.02):
    if prev_landmarks is None:
        return False
    
    total_change = 0
    for current, previous in zip(landmarks, prev_landmarks):
        total_change += np.linalg.norm(np.array([current.x, current.y]) - np.array([previous.x, previous.y]))
    
    average_change = total_change / len(landmarks)
    
    return average_change < threshold

# Function to draw the stability status on the image
def draw_stability_status(image, status):
    cv2.putText(image, f'Status: {status}', (10, image.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

# Initialize variables for stability detection
prev_landmarks = None
stability_status = "Moving"

# Get the start time
start_time = time.time()

# Get the current time for window close check
window_start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Perform pose detection
    results = pose.process(image)
    
    # Revert the image to BGR for rendering
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.pose_landmarks:
        # Extract landmarks
        landmarks = results.pose_landmarks.landmark
        
        # Determine stability
        if is_body_stable(landmarks, prev_landmarks):
            stability_status = "Stable"
        else:
            stability_status = "Moving"
        
        # Update previous landmarks
        prev_landmarks = landmarks
        
        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Draw body part labels
        body_parts = {
            'Hip': [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
            'Knee': [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
            'Ankle': [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
            'Shoulder': [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            'Elbow': [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
            'Wrist': [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        }
        
        for part, coord in body_parts.items():
            coord = tuple(np.multiply(coord, [image.shape[1], image.shape[0]]).astype(int))
            cv2.putText(image, part, coord, cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Draw the stability status on the image
        draw_stability_status(image, stability_status)
    
    # Display the image
    cv2.imshow('Pose Detection', image)
    
    # Check if 'q' key was pressed or window was closed
    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        break
    
    # Check elapsed time for the video window and stop after 30 seconds
    window_elapsed_time = time.time() - window_start_time
    if window_elapsed_time > 30:
        print("Window time's up! Closing the video window.")
        break
    
    # Check total elapsed time and stop the entire program after 100 seconds
    elapsed_time = time.time() - start_time
    if elapsed_time > 100:
        print("Kernel time's up! Stopping the program.")
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Terminate the script after 100 seconds
if elapsed_time > 100:
    print("Exiting the script.")
    exit()  # This will close the Python script if running in a standalone mode

Window time's up! Closing the video window.


In [3]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe Pose model for multi-person pose estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# Video capture (0 for webcam or use a video file path)
cap = cv2.VideoCapture(0)

# Colors for different persons (BGR format)
colors = [(255, 0, 0), (0, 255, 0)]  # Blue and Green for two persons